In [168]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

To start this, we need to differentiate between all of the scrapped data that we have gotten.

1. MVPS - Tracks MVP data, including MVP shares, votes, etc
2. Players - Tracks the Advanced stats of NBA players, like WS, WS/48, OBPM, DBPM, etc
3. Normal_Stats - Tracks normal stats, like PTS, BLKS, REBS, ASTS, etc
4. Record - Shows records of teams that players are on
5. Abbreviation - Abbreviation of Team Names so we can connect the dataframes

In [220]:
players = pd.read_csv('files/scrapped_csv/advanced_stats.csv')
mvps = pd.read_csv('files/scrapped_csv/mvps.csv')
normal_stats = pd.read_csv('files/scrapped_csv/player_stats.csv')
record = pd.read_csv('files/scrapped_csv/team_record.csv')
abbreviation = pd.read_csv('files/scrapped_csv/nba_abbreviations.csv')

In [172]:
players

,Unnamed: 0,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Year
0,0,1,Alaa Abdelnaby,PF,22,POR,43,290,13.1,.499,...,0.0,0.5,0.5,.079,NaN,-3.4,-1.2,-4.6,-0.2,1991
1,1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,1505,12.2,.448,...,-0.7,-0.3,-1.0,-0.031,NaN,-2.0,-3.0,-5.0,-1.1,1991
2,2,3,Mark Acres,C,28,ORL,68,1313,9.2,.551,...,1.4,1.1,2.5,.090,NaN,-2.8,-0.2,-3.0,-0.3,1991
3,3,4,Michael Adams,PG,28,DEN,66,2346,22.3,.530,...,5.8,0.4,6.3,.128,NaN,6.0,-0.7,5.3,4.3,1991
4,4,5,Mark Aguirre,SF,31,DET,78,2006,16.7,.526,...,2.8,2.7,5.5,.132,NaN,1.2,0.2,1.4,1.7,1991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18880,836,601,Thaddeus Young,PF,33,TOR,26,475,15.8,.526,...,0.5,0.8,1.3,.127,NaN,-0.2,2.2,2.0,0.5,2022
18881,837,602,Trae Young,PG,23,ATL,76,2652,25.4,.603,...,9.0,1.0,10.0,.181,NaN,7.1,-2.0,5.2,4.8,2022
18882,838,603,Omer Yurtseven,C,23,MIA,56,706,17.4,.546,...,0.8,1.4,2.1,.145,NaN,-1.4,0.4,-1.0,0.2,2022
18883,839,604,Cody Zeller,C,29,POR,27,355,17.2,.627,...,0.9,0.2,1.1,.143,NaN,-1.2,-1.0,-2.1,0.0,2022


For the players.csv file, get rid of unnecessary data as well as the rows in which the scrapping couldn't get rid of(ie player, team, etc). Additionally, since we want to clean this data, we have to make sure that there aren't duplicate teams and fix problems that might occur due to some data being incorrectly formatted.

In [173]:
def total_stats(df):
    if df.shape[0] == 1:
        return df
    else:
        row = df[df.get('Tm') == 'TOT']
        row['Tm'] = df.iloc[-1,:]['Tm']
        return row
    
advanced_stats = players.groupby(['Player', 'Year']).apply(total_stats)

At this point, as we have been able to look at all of the full player stats, we want to make sure each player's stats for each season is correct. To do so, we adjust for the fact that many players get traded each season. Fortunately, Basketball Reference has a team marker marked "TOT", which accounts for the total average of the players stats if they were traded. We then only take that player's stats and mark them for the team that they started the season with

In [174]:
advanced_stats

Unnamed: 0   Rk          Player Pos Age   Tm   G  \
Player         Year                                                          
A.C. Green     1991 164          164  135      A.C. Green  PF  27  LAL  82   
               1992 633          174  141      A.C. Green  PF  28  LAL  82   
               1993 1092         157  137      A.C. Green  PF  29  LAL  82   
               1994 1579         177  149      A.C. Green  PF  30  PHO  82   
               1995 2067         165  142      A.C. Green  SF  31  PHO  82   
...                              ...  ...             ...  ..  ..  ...  ..   
Željko Rebrača 2002 6095         394  336  Željko Rebrača   C  29  DET  74   
               2003 6595         374  316  Željko Rebrača   C  30  DET  30   
               2004 7176         452  338  Željko Rebrača   C  31  ATL  24   
               2005 7776         446  350  Željko Rebrača   C  32  LAC  58   
               2006 8370         433  344  Željko Rebrača   C  33  LAC  29   

                            MP   PER   TS%  ...  OWS  DWS   WS WS/48  \
Player         Year                         ...                        
A.C. Green     1991 164   2164  13.8  .556  ...  3.2  2.9  6.1  .135   
               1992 633   2902  16.7  .556  ...  5.8  2.9  8.8  .145   
               1993 1092  2819  16.3  .603  ...  6.1  2.6  8.6  .147   
               1994 1579  2825  17.0  .555  ...  6.7  2.6  9.3  .157   
               1995 2067  2687  14.2  .596  ...  4.6  2.1  6.7  .120   
...                        ...   ...   ...  ...  ...  ...  ...   ...   
Željko Rebrača 2002 6095  1179  16.2  .569  ...  1.4  1.9  3.3  .134   
               2003 6595   488  14.0  .596  ...  0.7  0.7  1.3  .133   
               2004 7176   273  10.5  .504  ...  0.1  0.4  0.5  .097   
               2005 7776   928  14.0  .625  ...  1.4  0.9  2.4  .122   
               2006 8370   412  10.7  .592  ...  0.2  0.5  0.7  .086   

                         Unnamed: 24  OBPM  DBPM   BPM  VORP  Year  
Player         Year                                                 
A.C. Green     1991 164          NaN   0.1  -0.2   0.0   1.1  1991  
               1992 633          NaN   1.7  -1.0   0.7   2.0  1992  
               1993 1092         NaN   1.6  -0.9   0.7   1.9  1993  
               1994 1579         NaN   1.8  -1.2   0.6   1.8  1994  
               1995 2067         NaN   0.8  -0.9  -0.1   1.3  1995  
...                              ...   ...   ...   ...   ...   ...  
Željko Rebrača 2002 6095         NaN  -2.9   0.6  -2.3  -0.1  2002  
               2003 6595         NaN  -2.8   0.2  -2.6  -0.1  2003  
               2004 7176         NaN  -4.5   0.6  -3.9  -0.1  2004  
               2005 7776         NaN  -1.7   0.4  -1.3   0.2  2005  
               2006 8370         NaN  -3.7   0.9  -2.8  -0.1  2006  

[14697 rows x 31 columns]

Here, since everything is already grouped up by the 'Player' and 'Year' columns, we need to drop the index level in order to get data that can be accessed by the player and the year

In [175]:
advanced_stats.index = advanced_stats.index.droplevel()

In [176]:
advanced_stats

,,Unnamed: 0,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Year
Year,,,,,,,,,,,,,,,,,,,,,,
1991,164,164,135,A.C. Green,PF,27,LAL,82,2164,13.8,.556,...,3.2,2.9,6.1,.135,NaN,0.1,-0.2,0.0,1.1,1991
1992,633,174,141,A.C. Green,PF,28,LAL,82,2902,16.7,.556,...,5.8,2.9,8.8,.145,NaN,1.7,-1.0,0.7,2.0,1992
1993,1092,157,137,A.C. Green,PF,29,LAL,82,2819,16.3,.603,...,6.1,2.6,8.6,.147,NaN,1.6,-0.9,0.7,1.9,1993
1994,1579,177,149,A.C. Green,PF,30,PHO,82,2825,17.0,.555,...,6.7,2.6,9.3,.157,NaN,1.8,-1.2,0.6,1.8,1994
1995,2067,165,142,A.C. Green,SF,31,PHO,82,2687,14.2,.596,...,4.6,2.1,6.7,.120,NaN,0.8,-0.9,-0.1,1.3,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,6095,394,336,Željko Rebrača,C,29,DET,74,1179,16.2,.569,...,1.4,1.9,3.3,.134,NaN,-2.9,0.6,-2.3,-0.1,2002
2003,6595,374,316,Željko Rebrača,C,30,DET,30,488,14.0,.596,...,0.7,0.7,1.3,.133,NaN,-2.8,0.2,-2.6,-0.1,2003
2004,7176,452,338,Željko Rebrača,C,31,ATL,24,273,10.5,.504,...,0.1,0.4,0.5,.097,NaN,-4.5,0.6,-3.9,-0.1,2004


In [177]:
advanced_stats.index = advanced_stats.index.droplevel()

In [178]:
advanced_stats

,Unnamed: 0,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Year
164,164,135,A.C. Green,PF,27,LAL,82,2164,13.8,.556,...,3.2,2.9,6.1,.135,NaN,0.1,-0.2,0.0,1.1,1991
633,174,141,A.C. Green,PF,28,LAL,82,2902,16.7,.556,...,5.8,2.9,8.8,.145,NaN,1.7,-1.0,0.7,2.0,1992
1092,157,137,A.C. Green,PF,29,LAL,82,2819,16.3,.603,...,6.1,2.6,8.6,.147,NaN,1.6,-0.9,0.7,1.9,1993
1579,177,149,A.C. Green,PF,30,PHO,82,2825,17.0,.555,...,6.7,2.6,9.3,.157,NaN,1.8,-1.2,0.6,1.8,1994
2067,165,142,A.C. Green,SF,31,PHO,82,2687,14.2,.596,...,4.6,2.1,6.7,.120,NaN,0.8,-0.9,-0.1,1.3,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6095,394,336,Željko Rebrača,C,29,DET,74,1179,16.2,.569,...,1.4,1.9,3.3,.134,NaN,-2.9,0.6,-2.3,-0.1,2002
6595,374,316,Željko Rebrača,C,30,DET,30,488,14.0,.596,...,0.7,0.7,1.3,.133,NaN,-2.8,0.2,-2.6,-0.1,2003
7176,452,338,Željko Rebrača,C,31,ATL,24,273,10.5,.504,...,0.1,0.4,0.5,.097,NaN,-4.5,0.6,-3.9,-0.1,2004
7776,446,350,Željko Rebrača,C,32,LAC,58,928,14.0,.625,...,1.4,0.9,2.4,.122,NaN,-1.7,0.4,-1.3,0.2,2005


In [179]:
advanced_stats.columns

Index(['Unnamed: 0', 'Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48',
       'Unnamed: 24', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Year'],
      dtype='object')

In [180]:
advanced_stats.drop(columns = ['Unnamed: 0', 'Rk', 'Unnamed: 19', 'Unnamed: 24'], inplace=True)

In [181]:
players = advanced_stats.sort_values('Year', ascending = True)

Get rid of players who aren't actually players(web scrapping didn't get rid of certain rows), and remove unneccessary symbols in the players names in order to merge correctly with the 'mvps' dataframe. Also get rid of players who only played for multiple teams in one season(only take the players with TOT)

In [182]:
players['Player'] = players['Player'].apply(lambda x: x.strip('*'))
players = players.apply(pd.to_numeric, errors = 'ignore')
players.drop(columns = ['MP'], inplace = True)

,Player,Pos,Age,Tm,G,PER,TS%,3PAr,FTr,ORB%,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year
164,A.C. Green,PF,27,LAL,82,13.8,0.556,0.101,0.557,11.3,...,15.6,3.2,2.9,6.1,0.135,0.1,-0.2,0.0,1.1,1991
13,Ron Anderson,SF,32,PHI,82,15.5,0.524,0.041,0.188,5.0,...,23.2,2.3,1.8,4.1,0.085,-0.2,-1.4,-1.6,0.2,1991
35,Rolando Blackman,SG,31,DAL,80,16.6,0.545,0.087,0.248,2.4,...,24.2,4.4,1.0,5.5,0.089,1.4,-1.4,0.0,1.5,1991
265,Rodney McCray,PF,29,DAL,74,14.6,0.551,0.057,0.292,6.8,...,15.5,3.1,2.3,5.3,0.100,0.4,0.6,1.0,1.9,1991
310,Chuck Person,SF,26,IND,80,16.1,0.553,0.165,0.186,5.6,...,24.8,3.1,0.8,3.9,0.073,1.7,-2.4,-0.7,0.8,1991
45,Chucky Brown,SF,22,CLE,74,12.0,0.555,0.008,0.287,6.1,...,19.4,0.7,0.9,1.6,0.052,-1.8,-1.5,-3.4,-0.5,1991
386,Rod Strickland,PG,24,SAS,58,16.2,0.538,0.051,0.324,3.2,...,18.1,2.7,3.1,5.7,0.132,1.1,1.3,2.4,2.3,1991
240,Cliff Levingston,PF,30,CHI,78,11.5,0.488,0.014,0.323,11.7,...,15.7,0.7,1.4,2.0,0.095,-2.7,-0.1,-2.9,-0.2,1991
238,Clifford Lett,PG,25,SAS,7,6.9,0.516,0.034,0.310,1.2,...,17.2,-0.1,0.1,0.0,0.014,-5.1,-0.1,-5.2,-0.1,1991
176,Ron Harper,SG,27,LAC,39,14.5,0.463,0.203,0.298,4.4,...,28.0,-0.8,1.6,0.9,0.030,0.6,0.4,1.0,1.0,1991


With this, the 'players' DataFrame has been completely cleaned. Now, in order to merge it with the 'mvps', we need to clean 'mvps' as well. In the MVP file, get rid of the useless/already-known columns(ie, TRB, AST, BLK, FG%, 3p%, FT%, G, MP)

In [222]:
mvps = mvps[['Player', 'Rank', 'Pts Won', 'Pts Max', 'Share', 'Year']]
mvps

,Player,Rank,Pts Won,Pts Max,Share,Year
0,Michael Jordan,1,891.0,960,0.928,1991
1,Magic Johnson,2,497.0,960,0.518,1991
2,David Robinson,3,476.0,960,0.496,1991
3,Charles Barkley,4,222.0,960,0.231,1991
4,Karl Malone,5,142.0,960,0.148,1991
...,...,...,...,...,...,...
481,Stephen Curry,8,4.0,1000,0.004,2022
482,Chris Paul,9,2.0,1000,0.002,2022
483,DeMar DeRozan,10T,1.0,1000,0.001,2022
484,Kevin Durant,10T,1.0,1000,0.001,2022


In [185]:
mvps[mvps.Player == 'James Harden']

,Player,Pts Won,Pts Max,Share,Year
363,James Harden,33.0,1210,0.027,2013
376,James Harden,85.0,1250,0.068,2014
390,James Harden,936.0,1300,0.720,2015
409,James Harden,9.0,1310,0.007,2016
412,James Harden,753.0,1010,0.746,2017
422,James Harden,965.0,1010,0.955,2018
436,James Harden,776.0,1010,0.768,2019
449,James Harden,367.0,1010,0.363,2020
471,James Harden,1.0,1010,0.001,2021


Here, we combine the players and the mvps, using an outer join (in order to capture all of rows in both players and MVPs). This ensures that we still have every player and MVP in the bracket together in the correct .csv file.

In [186]:
combined = players.merge(mvps, how='outer')

A good way to make sure your merge worked correctly is to check how many rows we had before and after. If the number isn't the same, something wrong occurred.

In [187]:
combined

,Player,Pos,Age,Tm,G,PER,TS%,3PAr,FTr,ORB%,...,WS,WS/48,OBPM,DBPM,BPM,VORP,Year,Pts Won,Pts Max,Share
0,A.C. Green,PF,27,LAL,82,13.8,0.556,0.101,0.557,11.3,...,6.1,0.135,0.1,-0.2,0.0,1.1,1991,NaN,NaN,NaN
1,Ron Anderson,SF,32,PHI,82,15.5,0.524,0.041,0.188,5.0,...,4.1,0.085,-0.2,-1.4,-1.6,0.2,1991,NaN,NaN,NaN
2,Rolando Blackman,SG,31,DAL,80,16.6,0.545,0.087,0.248,2.4,...,5.5,0.089,1.4,-1.4,0.0,1.5,1991,NaN,NaN,NaN
3,Rodney McCray,PF,29,DAL,74,14.6,0.551,0.057,0.292,6.8,...,5.3,0.100,0.4,0.6,1.0,1.9,1991,NaN,NaN,NaN
4,Chuck Person,SF,26,IND,80,16.1,0.553,0.165,0.186,5.6,...,3.9,0.073,1.7,-2.4,-0.7,0.8,1991,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14692,Kyle Lowry,PG,35,MIA,63,15.0,0.600,0.609,0.278,1.8,...,6.4,0.143,1.1,0.8,2.0,2.1,2022,NaN,NaN,NaN
14693,Steven Adams,C,28,MEM,76,17.6,0.560,0.003,0.518,17.9,...,6.8,0.163,1.0,1.0,2.0,2.0,2022,NaN,NaN,NaN
14694,Kyle Kuzma,PF,26,WAS,66,15.2,0.547,0.401,0.234,3.6,...,2.0,0.044,0.2,-0.4,-0.2,1.0,2022,NaN,NaN,NaN
14695,Tyler Johnson,PG,29,SAS,6,7.1,0.425,0.650,0.000,3.5,...,0.1,0.033,-3.9,0.8,-3.0,0.0,2022,NaN,NaN,NaN


In case there is rows of players who have empty rows, we want to make sure we fill in their information with 0s, because they didn't receive any MVP shares or votes at all, meaning they had 0 points

In [189]:
combined[['Pts Won', 'Pts Max', 'Share']] = combined[['Pts Won', 'Pts Max', 'Share']].fillna(0)

Now, let's move on to team record and how it affects the actual player. First, remove the useless columns and the useless rows that were not taken out by the Data Scrapping. This includes records in the months of the year, since the NBA didn't start tracking that 

In [191]:
record

,Unnamed: 0,Rk,Team,Overall,Home,Road,E,W,A,C,...,Mar,Apr,Year,Oct,May,SE,NW,SW,Jul,Aug
0,0,1,Portland Trail Blazers,63-19,36-5,27-14,21-5,42-14,10-2,11-3,...,9-5,10-1,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,Chicago Bulls,61-21,35-6,26-15,42-12,19-9,17-7,25-5,...,13-4,8-3,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,Los Angeles Lakers,58-24,33-8,25-16,18-8,40-16,9-3,9-5,...,10-6,7-3,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,Boston Celtics,56-26,35-6,21-20,35-19,21-7,20-6,15-13,...,11-5,4-6,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,Phoenix Suns,55-27,32-9,23-18,20-6,35-21,9-3,11-3,...,13-3,6-5,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,26,26,Indiana Pacers,25-57,16-25,9-32,11-41,14-16,5-13,2-14,...,4-10,0-5,2022,1-6,NaN,4-14,4-6,6-4,NaN,NaN
950,27,27,Oklahoma City Thunder,24-58,12-29,12-29,7-23,17-35,4-6,2-8,...,3-12,2-4,2022,1-5,NaN,1-9,6-10,6-12,NaN,NaN
951,28,28,Detroit Pistons,23-59,13-28,10-31,18-34,5-25,5-13,6-10,...,6-10,2-3,2022,1-5,NaN,7-11,2-8,2-8,NaN,NaN
952,29,29,Orlando Magic,22-60,12-29,10-31,12-40,10-20,4-14,5-13,...,5-10,2-3,2022,1-6,NaN,3-13,6-4,3-7,NaN,NaN


In [192]:
del record['Unnamed: 0']
del record['Rk']
record = record.drop(['Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Oct', 'SE', 'NW', 'SW', 'Jul', 'Aug', 'Pre', 'Post', '≤3', '≥10', 'C', 'M', 'P', 'E', 'W', "A"], axis = 1)

Some of the teams are unique in the system, because they either weren't configured correctly or had teams that had different names before; for instance, some players had Team called 'Team' or had teams that aren't currently in the NBA. Some examples include the 'New Orleans/Oklahoma City Hornets' for when the New Orleans Hornets (now the New Orlean Pelicans) played in Oklahoma City due to Hurricane Katrina. Others include the Charlotte Bobcats, which officially became the Charlotte Hornets in the 2014-2015 season. 

In [193]:
record.Team.unique()

array(['Portland Trail Blazers', 'Chicago Bulls', 'Los Angeles Lakers',
       'Boston Celtics', 'Phoenix Suns', 'San Antonio Spurs', 'Utah Jazz',
       'Houston Rockets', 'Detroit Pistons', 'Milwaukee Bucks',
       'Golden State Warriors', 'Philadelphia 76ers', 'Atlanta Hawks',
       'Indiana Pacers', 'Seattle SuperSonics', 'New York Knicks',
       'Cleveland Cavaliers', 'Los Angeles Clippers', 'Orlando Magic',
       'Washington Bullets', 'Minnesota Timberwolves', 'Dallas Mavericks',
       'Charlotte Hornets', 'New Jersey Nets', 'Sacramento Kings',
       'Miami Heat', 'Denver Nuggets', 'Toronto Raptors',
       'Vancouver Grizzlies', 'Washington Wizards', 'Memphis Grizzlies',
       'New Orleans Hornets', 'Team', 'Charlotte Bobcats',
       'New Orleans/Oklahoma City Hornets', 'Oklahoma City Thunder',
       'Brooklyn Nets', 'New Orleans Pelicans'], dtype=object)

In [194]:
record = record[record['Team'] != 'Team']

Separate the Wins and Losses so our Data is easy to use and read

In [195]:
record['Wins'] = record['Overall'].apply(lambda x: int(x.split('-')[0]))
record['Losses'] = record['Overall'].apply(lambda x: int(x.split('-')[1]))

In [196]:
record['W/L%'] = round(record['Wins']/(record['Wins'] + record['Losses']), 3)

From the previous dataframe of NBA Team abbreviations, we can now map our teams to their nicknames, because it creates less confusion and an easier time to look at each player in order to merge in the future

In [213]:
nicknames = dict()
for i in range(len(abbreviation.abv)):
    nicknames[abbreviation.abv.iloc[i]] = abbreviation.team.iloc[i]

In [214]:
combined['Team'] = combined['Tm'].map(nicknames)

Another metric that can be important in determining how likely a player is to get MVP shares is Game Score, from John Hollinger. Calculates how well a player did in their games, with 30-40 being a great game and 10 being an average game

In [225]:
def game_score_formula(pts, fg, fga, fta, ft, orb, drb, stl, ast, blk, pf, tov):
    """ 
    A measure of how well a player is doing in a single game.
    Created by John Hollinger to give rough measure of how well each
    singular player did in a game(40 is a great game, 10 is average)
    """
    return (pts) + (0.4 * fg) - (0.7 * fga) + (0.4 * (fta-ft)) + (0.7 * orb) + (0.3 * drb) + stl + (0.7 * ast) + (0.7 * blk) - (0.4 * pf) - tov

In [226]:
normal_stats.columns

Index(['Unnamed: 0', 'Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'Year'],
      dtype='object')

In [231]:
normal_stats.drop(columns = ['Unnamed: 0', 'Rk'], inplace = True)

In [241]:
normal_stats = normal_stats.apply(pd.to_numeric, errors = 'ignore')

Get rid of other columns that aren't necessary for testing the data: Overall, Home, Road

For a lot of the columns in our dataframe, most seem to be objects stored as strings within pandas. Since that doesn't work well with the model we have, we want to convert the needed ones into numeric mode to work well

In [260]:
normal_stats = normal_stats[normal_stats.Player != 'Player']
normal_stats['Player'] = normal_stats['Player'].apply(lambda x: x.strip('*'))
normal_stats_grouped = normal_stats.groupby(['Player', 'Year']).apply(total_stats)

In [261]:
normal_stats_grouped.index = normal_stats_grouped.index.droplevel()
normal_stats_grouped.index = normal_stats_grouped.index.droplevel()

In [262]:
normal_stats_grouped[normal_stats_grouped.Player == 'James Harden']

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
10495,James Harden,SG,20,OKC,76,0,22.9,3.1,7.6,.403,...,0.6,2.6,3.2,1.8,1.1,0.3,1.4,2.6,9.9,2010
11149,James Harden,SG,21,OKC,82,5,26.7,3.6,8.3,.436,...,0.5,2.6,3.1,2.1,1.1,0.3,1.3,2.5,12.2,2011
11742,James Harden,SG,22,OKC,62,2,31.4,5.0,10.1,.491,...,0.5,3.6,4.1,3.7,1.0,0.2,2.2,2.4,16.8,2012
12310,James Harden,SG,23,HOU,78,78,38.3,7.5,17.1,.438,...,0.8,4.1,4.9,5.8,1.8,0.5,3.8,2.3,25.9,2013
12950,James Harden,SG,24,HOU,73,73,38.0,7.5,16.5,.456,...,0.8,3.9,4.7,6.1,1.6,0.4,3.6,2.4,25.4,2014
13584,James Harden,SG,25,HOU,81,81,36.8,8.0,18.1,.440,...,0.9,4.7,5.7,7.0,1.9,0.7,4.0,2.6,27.4,2015
14211,James Harden,SG,26,HOU,82,82,38.1,8.7,19.7,.439,...,0.8,5.3,6.1,7.5,1.7,0.6,4.6,2.8,29.0,2016
14816,James Harden,PG,27,HOU,81,81,36.4,8.3,18.9,.440,...,1.2,7.0,8.1,11.2,1.5,0.5,5.7,2.7,29.1,2017
15457,James Harden,SG,28,HOU,72,72,35.4,9.0,20.1,.449,...,0.6,4.8,5.4,8.8,1.8,0.7,4.4,2.3,30.4,2018
16184,James Harden,PG,29,HOU,78,78,36.8,10.8,24.5,.442,...,0.8,5.8,6.6,7.5,2.0,0.7,5.0,3.1,36.1,2019


In [263]:
merged

,Player,Pos_x,Age_x,Tm_x,G_x,PER,TS%,3PAr,FTr,ORB%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,A.C. Green,PF,27.0,LAL,82.0,13.8,0.556,0.101,0.557,11.3,...,.738,2.5,3.8,6.3,0.9,0.7,0.3,1.2,1.4,9.1
1,Ron Anderson,SF,32.0,PHI,82.0,15.5,0.524,0.041,0.188,5.0,...,.833,1.3,3.2,4.5,1.4,0.8,0.2,1.2,2.0,14.6
2,Rolando Blackman,SG,31.0,DAL,80.0,16.6,0.545,0.087,0.248,2.4,...,.865,0.8,2.4,3.2,3.8,0.9,0.2,2.0,1.9,19.9
3,Rodney McCray,PF,29.0,DAL,74.0,14.6,0.551,0.057,0.292,6.8,...,.803,2.1,5.5,7.6,3.5,0.9,0.7,1.7,2.7,11.4
4,Chuck Person,SF,26.0,IND,80.0,16.1,0.553,0.165,0.186,5.6,...,.721,1.5,3.7,5.2,3.0,0.7,0.2,2.3,2.8,18.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15277,Šarūnas Marčiulionis*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,.788,0.9,1.9,2.9,3.4,1.6,0.1,2.7,3.3,18.9
15278,Šarūnas Marčiulionis*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,.761,1.3,1.9,3.2,3.5,1.7,0.1,2.5,3.1,17.4
15279,Šarūnas Marčiulionis*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,.732,0.3,0.8,1.0,1.7,1.1,0.0,1.5,1.9,9.3
15280,Šarūnas Marčiulionis*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,.775,0.4,1.1,1.5,2.2,1.0,0.1,1.8,2.1,10.8


In [265]:
merged = combined.merge(normal_stats_grouped, how = 'outer', on = ['Player', 'Year'])

In [266]:
merged

,Player,Pos_x,Age_x,Tm_x,G_x,PER,TS%,3PAr,FTr,ORB%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,A.C. Green,PF,27,LAL,82,13.8,0.556,0.101,0.557,11.3,...,.738,2.5,3.8,6.3,0.9,0.7,0.3,1.2,1.4,9.1
1,Ron Anderson,SF,32,PHI,82,15.5,0.524,0.041,0.188,5.0,...,.833,1.3,3.2,4.5,1.4,0.8,0.2,1.2,2.0,14.6
2,Rolando Blackman,SG,31,DAL,80,16.6,0.545,0.087,0.248,2.4,...,.865,0.8,2.4,3.2,3.8,0.9,0.2,2.0,1.9,19.9
3,Rodney McCray,PF,29,DAL,74,14.6,0.551,0.057,0.292,6.8,...,.803,2.1,5.5,7.6,3.5,0.9,0.7,1.7,2.7,11.4
4,Chuck Person,SF,26,IND,80,16.1,0.553,0.165,0.186,5.6,...,.721,1.5,3.7,5.2,3.0,0.7,0.2,2.3,2.8,18.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14692,Kyle Lowry,PG,35,MIA,63,15.0,0.600,0.609,0.278,1.8,...,.851,0.5,4.0,4.5,7.5,1.1,0.3,2.7,2.8,13.4
14693,Steven Adams,C,28,MEM,76,17.6,0.560,0.003,0.518,17.9,...,.543,4.6,5.4,10.0,3.4,0.9,0.8,1.5,2.0,6.9
14694,Kyle Kuzma,PF,26,WAS,66,15.2,0.547,0.401,0.234,3.6,...,.712,1.1,7.4,8.5,3.5,0.6,0.9,2.6,1.9,17.1
14695,Tyler Johnson,PG,29,SAS,6,7.1,0.425,0.650,0.000,3.5,...,NaN,0.5,1.5,2.0,1.2,0.5,0.5,0.3,1.2,2.8


In [269]:
merged.to_csv('files/FINAL_STATS/final_player_stats.csv')